In [14]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
import plotly.express as px
import folium
from folium import plugins
from folium.plugins import HeatMap

ModuleNotFoundError: No module named 'geopandas'

In [7]:
df = pd.read_csv('../../Data/FuentesOriginales/Kaggle_OCDE/population-cleaned.csv')
df.head()

,Name,ASCII Name,Alternate Names,Country Code,Country name EN,Population,Timezone,LABEL EN,Latitude,Longitude
0,Pizhanka,Pizhanka,"Pizhanka,Pizjanka,Пижанка",RU,Russian Federation,3900.0,Europe/Kirov,Russian Federation,57.46042,48.54200
1,Pitelino,Pitelino,"Petelino,Pitelino,Пителино",RU,Russian Federation,2355.0,Europe/Moscow,Russian Federation,54.57768,41.81448
2,Petro-Slavyanka,Petro-Slavyanka,"Petro-Slav'anka,Petro-Slavjanka,Petro-Slavyank...",RU,Russian Federation,1308.0,Europe/Moscow,Russian Federation,59.80222,30.50694
3,Palkino,Palkino,"Palkino,Палкино,Պալկինո",RU,Russian Federation,3112.0,Europe/Moscow,Russian Federation,57.54089,28.01257
4,Orlovka,Orlovka,"Orlovka,Орловка",RU,Russian Federation,3236.0,Europe/Moscow,Russian Federation,44.69139,44.17444


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140632 entries, 0 to 140631
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Name             140631 non-null  object 
 1   ASCII Name       140630 non-null  object 
 2   Alternate Names  116241 non-null  object 
 3   Country Code     140587 non-null  object 
 4   Country name EN  140464 non-null  object 
 5   Population       140632 non-null  float64
 6   Timezone         140632 non-null  object 
 7   LABEL EN         140464 non-null  object 
 8   Latitude         140632 non-null  float64
 9   Longitude        140632 non-null  float64
dtypes: float64(3), object(7)
memory usage: 7.0+ MB


In [13]:
def load_gpd_world():
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    # Fix missing country codes in data...
    world.loc[world['name'] == 'France', 'iso_a3'] = 'FRA'
    world.loc[world['name'] == 'Norway', 'iso_a3'] = 'NOR'
    world.loc[world['name'] == 'Somaliland', 'iso_a3'] = 'SOM'
    world.loc[world['name'] == 'Kosovo', 'iso_a3'] = 'RKS'
    
    return world

world = load_gpd_world()

NameError: name 'gpd' is not defined

In [12]:
# Clean the Coordinate column name
cities_population = cities_population.rename(columns={cities_population.columns[-1]:'Coordinate'})
# Only keep cities with a Coordinate
cities_population = cities_population.loc[~cities_population.Coordinate.isna()]
cities_population.columns = [col.replace(' ', '_') for col in cities_population.columns]
# Transform to Numeric
cities_population['Population'] = pd.to_numeric(cities_population['Population'])
cities_population['Elevation'] = pd.to_numeric(cities_population['Elevation'], errors='coerce')
cities_population['DIgital_Elevation_Model'] = pd.to_numeric(cities_population['DIgital_Elevation_Model'], errors='coerce')
# 'NA' for Namibia interpreted as NaN...
cities_population.loc[cities_population['Country_name_EN'] == 'Namibia', 'Country_Code'] = 'NA'
# remove spaces from column names
## only keep whats might be relevant for us
cities_population = cities_population[['Name', 'Country_Code', 'Country_name_EN', 'Population', 'Elevation', 'DIgital_Elevation_Model', 'Modification_date', 'Coordinate']]

NameError: name 'cities_population' is not defined

In [ ]:
# Remove all trailing comma from coordinate String
cities_population['Coordinate'] = cities_population['Coordinate'].apply(lambda x: [coordinate for coordinate in x.split(',') if len(coordinate) > 0])

# Extract longitude and latitude
long_lat_values = cities_population['Coordinate'].apply(pd.Series)
long_lat_values.columns = ['Latitude','Longitude']

# Append to dataframe
cities_population = cities_population.merge(long_lat_values, left_index=True, right_index=True)
cities_population = cities_population.drop(columns='Coordinate')

# Transform to geopandas
cities_population = gpd.GeoDataFrame(
        cities_population, 
        geometry=gpd.points_from_xy(cities_population.Longitude, cities_population.Latitude),
        crs='EPSG:4326'
)